In [ ]:
import sys
!{sys.executable} -m pip install pandas scikit-learn matplotlib

In [100]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split

In [108]:
x = pd.read_csv('../data/game_data_v2.csv')
x.head(3)

,away_team,away_score,away_team_xwoba_diff,away_lineup_xwoba,away_team_xfip,away_starter_xfip,away_close_win_pct,away_run_diff,home_team,home_score,home_team_xwoba_diff,home_lineup_xwoba,home_team_xfip,home_starter_xfip,home_close_win_pct,home_run_diff
0,SF,5,-0.01395,0.324867,4.3175,3.7785,0.481528,-50.85,AZ,6,0.00290,0.312183,4.0145,3.4820,0.543823,58.35
1,CHC,3,0.01790,0.320194,3.9005,3.6365,0.577740,151.35,STL,4,0.02085,0.313467,3.9985,3.6225,0.486630,68.65
2,NYY,3,0.02180,0.328811,3.8120,3.4600,0.485525,123.20,TB,7,0.00115,0.313050,4.2200,3.3140,0.430893,-15.95


In [109]:
# add output columns (scores) to separate dataset
y = pd.DataFrame({'away_score': x['away_score'], 'home_score': x['home_score']})
# y = pd.DataFrame({'home_win': x['home_score'] > x['away_score']})['home_win']

# remove output columns and team names from input dataset
x = x.drop(['away_score', 'home_score', 'away_team', 'home_team'], axis=1)
y.head(3)

0    True
1    True
2    True
Name: home_win, dtype: bool

In [120]:
# split data into training and testing
x_train, x_test, y_train, y_test = train_test_split(x, y, shuffle=True)

In [121]:
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, max_depth=15))\
# model = RandomForestClassifier(n_estimators=100, max_depth=15)
model.fit(x_train, y_train)

RandomForestClassifier(max_depth=15)

In [122]:
pred = model.predict(x_test)
pred[0:3]

array([ True,  True, False])

In [123]:
y_test.head(3)

11858    True
291      True
275      True
Name: home_win, dtype: bool

In [124]:
num_correct = 0
num_incorrect = 0

for i in range(len(pred)):
    away_score_pred = pred[i][0]
    home_score_pred = pred[i][1]
    away_score_actual = y_test['away_score'].iloc[i]
    home_score_actual = y_test['home_score'].iloc[i]

    pred_result = 1 if home_score_pred > away_score_pred else 0 # where 1 = home team wins
    actual_result = 1 if home_score_actual > away_score_actual else 0

    # pred_result = pred[i]
    # actual_result = y_test.iloc[i]
    
    if pred_result == actual_result:
        num_correct += 1
    else:
        num_incorrect += 1

print(f'Correct: {num_correct}')
print(f'Incorrect: {num_incorrect}')
print(f'W/L accuracy: {(num_correct / (num_correct + num_incorrect))*100}%')

Correct: 2202
Incorrect: 1668
W/L accuracy: 56.89922480620155%


In [79]:
model.score(x_test, y_test)

0.034759013311700726